In [29]:
from pydantic import BaseModel, Field
from typing import TypedDict,Annotated,Union,Dict
from langchain_ollama import ChatOllama
from langgraph.graph import START,StateGraph,END,MessagesState
from langgraph.graph.message import add_messages
from langchain_core.messages import HumanMessage,AnyMessage,BaseMessage
from langchain_core.runnables import RunnableConfig
import json
model = "jacob-ebey/phi4-tools:latest"
llm = ChatOllama(model=model,base_url="192.168.23.138:11439")

In [30]:
class State(TypedDict):
    messages : Annotated[list[BaseMessage], add_messages]
    CustomSystemPrompt : str
    


In [31]:
def SystemPromptCreator(state:State,config:RunnableConfig):
    return {'messages': [llm.invoke(state['messages'])]}


In [32]:
build = StateGraph(State)
build.add_node("prompt",SystemPromptCreator)

build.add_edge(START,"prompt")
build.add_edge("prompt",END)
graph = build.compile()

In [33]:
qurey ='''{ "experiments": [ { "title": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "objective": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "materials": [ "Operational Amplifier IC (LM741)", "Variable Resistor (10KΩ)", "Resistor (1KΩ)", "Input Signal Source (Function Generator)", "Connecting Wires", "Power Supply (9V)" ], "procedure": { "power_supply_connections": [ "Connect the positive terminal of the battery to pin 7 (V+) of the IC 741", "Connect the negative terminal of the battery to pin 4 (V-) of the IC 741" ], "input_signal_setup": [ "Attach one end of the potentiometer to the positive supply voltage", "Connect the wiper (middle terminal) of the potentiometer to pin 3 (non-inverting input) of the op-amp", "Connect the other end of the potentiometer to ground" ], "feedback_loop": [ "Connect a wire from pin 6 (output) directly to pin 2 (inverting input)" ], "load_resistor_connection": [ "Attach one terminal of the 1 kΩ resistor to pin 6 (output)", "Connect the other terminal of the resistor to ground" ], "biasing_and_stability": [ "Ensure pins 4 (V-) and 7 (V+) are properly connected for stable operation", "If necessary, connect a capacitor across V+ and V- for power supply decoupling" ], "testing_the_circuit": [ "Turn on the power supply and adjust the potentiometer to vary the input voltage", "Measure the output voltage at pin 6 with a voltmeter", "Verify that the output voltage matches the input voltage" ] }, "analysis": [ "Verify that the output voltage matches the input voltage, confirming the unity gain configuration", "Discuss the importance of negative feedback in achieving unity gain", "Analyze the effect of the variable resistor on the input voltage and output voltage", "Examine the role of the load resistor in the circuit and its impact on the output voltage" ] } ] } \n you are a experiment analysier based on the Ground truth above find out the key points to ensure that you judge other experiment with the above ground truth and make a system prompt out of it'''
inputs = {"messages": [("user", qurey)]}
graph.invoke({"messages":qurey,"CustomSystemPrompt":""})
# print(graph.state["CustomSystemPrompt"])

{'messages': [HumanMessage(content='{ "experiments": [ { "title": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "objective": "To set up and test a unity gain amplifier using an IC 741 operational amplifier", "materials": [ "Operational Amplifier IC (LM741)", "Variable Resistor (10KΩ)", "Resistor (1KΩ)", "Input Signal Source (Function Generator)", "Connecting Wires", "Power Supply (9V)" ], "procedure": { "power_supply_connections": [ "Connect the positive terminal of the battery to pin 7 (V+) of the IC 741", "Connect the negative terminal of the battery to pin 4 (V-) of the IC 741" ], "input_signal_setup": [ "Attach one end of the potentiometer to the positive supply voltage", "Connect the wiper (middle terminal) of the potentiometer to pin 3 (non-inverting input) of the op-amp", "Connect the other end of the potentiometer to ground" ], "feedback_loop": [ "Connect a wire from pin 6 (output) directly to pin 2 (inverting input)" ], "load_resistor_conne